In [1]:
from cv2 import aruco
import cv2
import numpy as np
import os
import csv
import sys
from tqdm import tqdm
from IPython.display import display, Image
import msgpack as mp
import msgpack_numpy as mpn
import matplotlib.pyplot as plt

In [2]:
_webcam_calib_folder = r"C:\Users\CMC\Documents\openposelibs\pose\armbo\recording_programs\test_data\omni_cam_9d0f_may_8_2023\calibration_00"
# _webcam_calib_folder = r"C:\Users\Sujith\Documents\Projects\armbo\recording_programs\test_data\omni_cam_9d0f_may_8_2023\calibration_00"

_webcam_calib_folder = os.path.join(_webcam_calib_folder)
_webcam_calib_pth = os.path.join( _webcam_calib_folder, "webcam_calibration.msgpack")

with open(_webcam_calib_pth, "rb") as f:
    webcam_calib = mp.Unpacker(f, object_hook=mpn.decode)
    _temp = next(webcam_calib)
    _webcam_cam_mat = _temp[0]
    _webcam_dist = _temp[1]
_webcam_video_pth = os.path.join(_webcam_calib_folder, "webcam_color.msgpack")
_webcam_timestamp_pth = os.path.join(_webcam_calib_folder, "webcam_timestamp.msgpack")

with open(os.path.join(_webcam_calib_folder, "webcam_rotmat.msgpack"), "rb") as f:
    webcam_rotmat = mp.Unpacker(f, object_hook=mpn.decode)
    _webcam_rot = next(webcam_rotmat)
    _webcam_org = next(webcam_rotmat)
print("camera matrix", _webcam_cam_mat)
print("distortion", _webcam_dist)

camera matrix [[880.19523452   0.         358.2789884 ]
 [  0.         881.87061144 242.70902376]
 [  0.           0.           1.        ]]
distortion [[ 1.24325941e-01 -1.20330000e-01 -2.99461015e-03 -1.15851300e-03
  -3.99283638e+00]]


In [3]:
ARUCO_PARAMETERS = aruco.DetectorParameters_create()
ARUCO_DICT = aruco.Dictionary_get(aruco.DICT_ARUCO_ORIGINAL)
markerLength = 0.05
markerSeparation = 0.01

board = aruco.GridBoard_create(
        markersX=1,
        markersY=1,
        markerLength=markerLength,
        markerSeparation=markerSeparation,
        dictionary=ARUCO_DICT)

Detect a ramdom image and show the rectangle

In [4]:
def add_noise(image):
    img = image[...,::-1]/255.0
    noise =  np.random.normal(loc=0, scale=1, size=img.shape)
    img2 = img*2
    n4 = np.clip(np.where(img2 <= 1, (img2*(1 + noise*0.4)), (1-img2+1)*(1 + noise*0.4)*-1 + 2)/2, 0,1)
    return n4


In [5]:
_folder_names = os.listdir(os.path.dirname(_webcam_calib_folder))
_folder_names.sort()
_folder_names = _folder_names[3:]

total_dummy_ids = []

for _folder_name in tqdm(_folder_names):
    _video_pth = os.path.join(os.path.dirname(_webcam_calib_folder), _folder_name, "webcam_color.msgpack")
    _video_file = open(_video_pth, "rb")
    _video = mp.Unpacker(_video_file, object_hook=mpn.decode)

    coord = {"x":[], "y":[], "z":[]}

    dummy_ids = []

    for frame_id, frame in enumerate(_video):
        # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = frame
        # estimate pose of aruco markers
        corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, ARUCO_DICT, parameters=ARUCO_PARAMETERS)
        # Refine detected markers
        # Eliminates markers not part of our board, adds missing markers to the board
        corners, ids, rejectedImgPoints, recoveredIds = aruco.refineDetectedMarkers(
                image = gray,
                board = board,
                detectedCorners = corners,
                detectedIds = ids,
                rejectedCorners = rejectedImgPoints,
                cameraMatrix = _webcam_cam_mat,
                distCoeffs = _webcam_dist)

        rotation_vectors, translation_vectors, _ = aruco.estimatePoseSingleMarkers(corners, 0.05, _webcam_cam_mat, _webcam_dist)

        if translation_vectors is None:
            coord["x"].append(np.nan)
            coord["y"].append(np.nan)
            coord["z"].append(np.nan)
            dummy_ids.append(frame_id)
            
        else:
            coord["x"].append(translation_vectors[0][0][0])
            coord["y"].append(translation_vectors[0][0][1])
            coord["z"].append(translation_vectors[0][0][2])

    for idx, val in enumerate(coord["z"]):
        if idx == 0:
            continue
        else:
            if abs(coord["z"][idx] - coord["z"][idx-1]) > 0.06:
                dummy_ids.append(idx)
            try:
                if abs(coord["z"][idx] - coord["z"][idx+1]) > 0.06:
                    dummy_ids.append(idx)
            except:
                pass

    for idx, val in enumerate(coord["x"]):
        if idx == 0:
            continue
        else:
            if abs(coord["x"][idx] - coord["x"][idx-1]) > 0.1:
                dummy_ids.append(idx)
            try:
                if abs(coord["x"][idx] - coord["x"][idx+1]) > 0.1:
                    dummy_ids.append(idx)
            except:
                pass

    total_dummy_ids.append(dummy_ids)

100%|██████████| 8/8 [03:05<00:00, 23.17s/it]


In [6]:
raw_data_pth = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
raw_data_pth = os.path.join(raw_data_pth, 'dataset', "raw_data")
if not os.path.exists(raw_data_pth):
    os.makedirs(os.path.join(raw_data_pth, "images"))
    os.makedirs(os.path.join(raw_data_pth, "labels"))

In [7]:
counter = 0
for _folder_id, _folder_name in tqdm(enumerate(_folder_names)):
    print(_folder_id, _folder_name)
    _video_pth = os.path.join(os.path.dirname(_webcam_calib_folder), _folder_name, "webcam_color.msgpack")
    _video_file = open(_video_pth, "rb")
    _video = mp.Unpacker(_video_file, object_hook=mpn.decode)
    height, width = next(_video).shape
    _video_file.close()
    _video_file = open(_video_pth, "rb")
    _video = mp.Unpacker(_video_file, object_hook=mpn.decode)

    fake_id = total_dummy_ids[_folder_id]

    for image_id, image in enumerate(_video):

        if image_id in fake_id:
            continue

        markerCorners, markerIds, rejected_img_points = aruco.detectMarkers(image, ARUCO_DICT, parameters=ARUCO_PARAMETERS)
        markerCorners, markerIds, rejectedImgPoints, recoveredIds = aruco.refineDetectedMarkers(
                image = image,
                board = board,
                detectedCorners = markerCorners,
                detectedIds = markerIds,
                rejectedCorners = rejected_img_points,
                cameraMatrix = _webcam_cam_mat,
                distCoeffs = _webcam_dist)
        
        if len(markerCorners) == 0:
            continue
            
        img_name = f"image{counter}.png"
        
        label_name = img_name.split(".")[0]
        label_path = os.path.join(raw_data_pth, "labels", f"{label_name}.txt")
        label_file = open(label_path, "w")
        label_writer = csv.writer(label_file, delimiter=" ")
        _markerCorners = markerCorners[0][0]

        bbox_x,bbox_y,bbox_width,bbox_height = cv2.boundingRect(_markerCorners)

        # normalize the values
        bbox_x = bbox_x / width
        bbox_y = bbox_y / height

        bbox_center_x = bbox_x + bbox_width / (2 * width)
        bbox_center_y = bbox_y + bbox_height / (2 * height)

        bbox_width = bbox_width / width
        bbox_height = bbox_height / height

        label_writer.writerow(["0", bbox_center_x, bbox_center_y, bbox_width, bbox_height,_markerCorners[0][0]/width, _markerCorners[0][1]/height, _markerCorners[1][0]/width, _markerCorners[1][1]/height, _markerCorners[2][0]/width, _markerCorners[2][1]/height, _markerCorners[3][0]/width, _markerCorners[3][1]/height, _markerCorners[0][0]/width, _markerCorners[0][1]/height])

        label_file.close()
        
        # save image
        image_path = os.path.join(raw_data_pth, "images", img_name)
        cv2.imwrite(image_path, image)

        counter += 1 # increment counter


0it [00:00, ?it/s]

0 sk36_30_forward_slow_00


1it [00:43, 43.75s/it]

1 sk36_30_forward_slow_01


2it [01:39, 50.53s/it]

...

6it [05:23, 57.55s/it]

6 sk36_30_sideways_00


7it [06:46, 65.75s/it]

7 sk36_30_sideways_01


8it [07:31, 56.46s/it]


In [8]:
_raw_saved_data_pth = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
_raw_saved_data_pth = os.path.join(_raw_saved_data_pth, 'dataset', "raw_data")
_raw_saved_data_pth = os.path.join(_raw_saved_data_pth, "images")
_raw_saved_data_list = os.listdir(_raw_saved_data_pth)

def gaussian_noise(image):

    gauss_noise=np.zeros(image.shape, dtype=np.uint8)
    cv2.randn(gauss_noise, 0, 5)
    gauss_noise=(gauss_noise*0.5).astype(np.uint8)
    gn_img = cv2.add(image,gauss_noise)
    return gn_img

def gaussian_blur(image):
    return cv2.GaussianBlur(image, (5, 5), 0)

#adding noise to twenty percent of the images
for img_name in tqdm(_raw_saved_data_list):
    _image_path = os.path.join(_raw_saved_data_pth, img_name)
    image = cv2.imread(_image_path)
    #adding noise to twenty percent of the images
    if np.random.random() < 0.2:
        image = gaussian_blur(image)
        _image_path = os.path.join(_raw_saved_data_pth, f"blur_{img_name}")
        cv2.imwrite(_image_path, image)

        label_name = img_name.split(".")[0]
        label_path = os.path.join(_raw_saved_data_pth, "..","labels", f"{label_name}.txt")
        label_file = open(label_path, "r")
        label_reader = csv.reader(label_file, delimiter=" ")
        label = list(label_reader)[0]
        label_file.close()

        label_path = os.path.join(os.path.join(_raw_saved_data_pth, "..", "labels"), f"blur_{img_name.split('.')[0]}.txt")
        label_file = open(label_path, "w")
        label_writer = csv.writer(label_file, delimiter=" ")
        label_writer.writerow(label)
        label_file.close()
    
    if np.random.random() > 0.2 and np.random.random()<0.4:
        image = add_noise(image)
        image = (image*255).astype(np.uint8)
        _image_path = os.path.join(_raw_saved_data_pth, f"noise_{img_name}")
        
        cv2.imwrite(_image_path, image)

        label_name = img_name.split(".")[0]
        label_path = os.path.join(_raw_saved_data_pth, "..","labels", f"{label_name}.txt")
        label_file = open(label_path, "r")
        label_reader = csv.reader(label_file, delimiter=" ")
        label = list(label_reader)[0]
        label_file.close()

        label_path = os.path.join(os.path.join(_raw_saved_data_pth, "..", "labels"), f"noise_{img_name.split('.')[0]}.txt")
        label_file = open(label_path, "w")
        label_writer = csv.writer(label_file, delimiter=" ")
        label_writer.writerow(label)
        label_file.close()


100%|██████████| 11987/11987 [21:11<00:00,  9.43it/s] 


In [14]:
# adding noise to the images

In [15]:
print(len(os.listdir(os.path.join(raw_data_pth, "labels"))), len(image_list))

2973 1963


Splitting it into traning and testing dataset based on COCOv8 format

In [9]:
# splitting dataset into train and validation and test
data_pth = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_pth = os.path.join(data_pth, 'dataset', "dataset_processed")

images_pth = os.path.join(data_pth, "images")
labels_pth = os.path.join(data_pth, "labels")

if not os.path.exists(images_pth):
    os.makedirs(os.path.join(images_pth, "train"))
    os.makedirs(os.path.join(images_pth, "val"))
    os.makedirs(os.path.join(images_pth, "test"))

if not os.path.exists(labels_pth):
    os.makedirs(os.path.join(labels_pth, "train"))
    os.makedirs(os.path.join(labels_pth, "val"))
    os.makedirs(os.path.join(labels_pth, "test"))
    
    
image_list = os.listdir(os.path.join(raw_data_pth, "images"))

# splitting dataset into train and test
for img_name in tqdm(image_list):
    
    _image_path = os.path.join(raw_data_pth, "images", img_name)
    image = cv2.imread(_image_path)
    
    label_name = img_name.split(".")[0]
    label_path = os.path.join(raw_data_pth, "labels", f"{label_name}.txt")
    label_file = open(label_path, "r")
    label_reader = csv.reader(label_file, delimiter=" ")
    label = list(label_reader)[0]
    label_file.close()
    
    if int(label[0]) == 0:
        if np.random.rand() < 0.7:
            # save image
            image_path = os.path.join(os.path.join(images_pth, "train"), img_name)
            cv2.imwrite(image_path, image)
            
            label_path = os.path.join(os.path.join(labels_pth, "train"), f"{label_name}.txt")
            label_file = open(label_path, "w")
            label_writer = csv.writer(label_file, delimiter=" ")
            label_writer.writerow(label)
            label_file.close()
            
        elif np.random.rand() < 0.9 and np.random.rand() > 0.7:
            # save image
            image_path = os.path.join(os.path.join(images_pth, "val"), img_name)
            cv2.imwrite(image_path, image)
            
            label_path = os.path.join(os.path.join(labels_pth, "val"), f"{label_name}.txt")
            label_file = open(label_path, "w")
            label_writer = csv.writer(label_file, delimiter=" ")
            label_writer.writerow(label)
            label_file.close()
            
        else:
            # save image
            image_path = os.path.join(os.path.join(images_pth, "test"), img_name)
            cv2.imwrite(image_path, image)
            
            label_path = os.path.join(os.path.join(labels_pth, "test"), f"{label_name}.txt")
            label_file = open(label_path, "w")
            label_writer = csv.writer(label_file, delimiter=" ")
            label_writer.writerow(label)
            label_file.close() 


100%|██████████| 18230/18230 [28:53<00:00, 10.52it/s]  
